In [ ]:
pip install transformers==4.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 51.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import math 
from sklearn.model_selection import train_test_split
import torch
import torch.nn
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

import logging
logging.basicConfig(level=logging.ERROR)
from torch.nn.utils.rnn import pad_sequence

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)# to use gpu if possible

cpu


In [ ]:
import json
import networkx as nx

# Read the JSONL file and parse the data
def read_fever_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

# Create the knowledge graph
def create_knowledge_graph(data):
    kg = nx.DiGraph()
    
    for item in data:
        claim_id = item['id']
        claim_text = item['claim']
        label = item['label']
        
        # Add claim node
        kg.add_node(claim_id, label="claim", text=claim_text)
        
        if label != "NOT ENOUGH INFO":
            for evidence_group in item['evidence']:
                for evidence in evidence_group:
                    evidence_id = evidence[1]
                    evidence_title = evidence[2]
                    evidence_sentence_num = evidence[3]
                    
                    # Add evidence node
                    kg.add_node(evidence_id, label="evidence", title=evidence_title, sentence_num=evidence_sentence_num)
                    
                    # Add edge between claim and evidence with the relationship label
                    kg.add_edge(claim_id, evidence_id, label=label)
    
    return kg

file_path = "/content/drive/MyDrive/NAM/train.jsonl"
data = read_fever_jsonl(file_path)
knowledge_graph = create_knowledge_graph(data)

In [ ]:
print("Nodes in the knowledge graph:", knowledge_graph.number_of_nodes())
print("Edges in the knowledge graph:", knowledge_graph.number_of_edges())

Nodes in the knowledge graph: 268910
Edges in the knowledge graph: 221476
